In [1]:
import pandas as pd
import numpy as np

In [35]:
data_base = pd.read_excel('./DATA_preprocess_EDA_base/big_merge_V2_meteo_SAT.xlsx').drop('Unnamed: 0', axis=1)

In [3]:
data_base.head()

,PARCELLE,LAT,LON,ALT,PRODREG,HT_VEG,DATE,SLOPE25,ASPECT25,ORIENTATION,...,TAVE_AVG,TAVE,TAVE_GROWTH,PRCP_S_S,PRCP_G_S,NDVI,EVI,NDMI,NDWI,DSWI
0,51,47.781623,8.612822,715.918970,1,3,1984-04-10,58.407726,67.342415,E,...,NaN,NaN,NaN,0.000000,0.000000,0.3779,0.0146,0.2264,-0.3325,0.4705
1,51,47.781623,8.612822,715.918970,1,3,1994-10-19,58.407726,67.342415,E,...,8.8859,9.026806,13.84776,117.278617,51.835496,0.4992,0.0160,0.2705,-0.4653,0.6865
2,51,47.781623,8.612822,715.918970,1,3,2011-11-09,58.407726,67.342415,E,...,9.0502,4.555499,14.64318,120.598667,57.801233,-0.1255,-0.0138,0.0107,0.1121,-0.0810
3,51,47.781623,8.612822,715.918970,1,3,2005-11-01,58.407726,67.342415,E,...,9.0457,3.006553,14.37088,122.999273,55.547082,0.6923,0.0272,0.2294,-0.6305,0.9579
4,384,47.736528,8.625154,563.829759,1,2,1995-07-10,55.683254,253.354935,O,...,8.9951,19.793490,13.83959,122.675652,55.827235,0.5552,0.0149,0.2017,-0.4864,0.6996


PREPROCESSING _ Code base for models temporal predictions

Ici, features engineering (création de nouvelles features à partir de la liste connues):

In [160]:
# Exemple, formule sécheresse...

### Choix TARGET et Features :

Target :

In [24]:
TARGET = ['TAUX_COUV_RAJ'] #exemple

Features :

In [59]:
# --- PAST --- 
# Attention : on peut, logiquement, inclure la TARGET... (connue dans le passé)
cat_strict = ['TAUX_COUV_RAJ','PRODREG','UNIT_VEG_GROS', 'TYPE_FORET305', 'ESPECE_DOM', 'TYP_RAJ_PPL', 'ORIENTATION'] #exemple
cat_ord_miss = ['TAILLE_PPL', 'MELANGE', 'HT_VEG', 'QUAL_STATION'] #exemple
numerics = ['ALT', 'TIGES_VIV_H', 'SURF_TER_HA', 'FEUILL_PER', 'CONIF_PER', 'PERF_CROI', 'NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI'] #exemple

# --- FUTURE KNOWN ---
# Attention : logiquement les features, potentiellement connues dans le futur ci-dessous sont aussi répertoriées ci-dessus dans le passé
add_cat_IFN_stable = ['PRODREG', 'ORIENTATION', 'UNIT_VEG_GROS']
add_cat_ord_IFN_stable = ['HT_VEG', 'QUAL_STATION']
add_IFN_numerics_stable = ['ALT'] #exemple
# données potentiellement connues car stables par parcelles :
#['LAT', 'LON', 'ALT', 'PRODREG', 'HT_VEG', 'SLOPE25', 'ASPECT25', 'ORIENTATION', 'PERF_CROI', 'QUAL_STATION', 'UNIT_VEG_FINE',
# 'UNIT_VEG_GROS', 'PROCESS_SILVA', 'PERI_CHENAUX', 'PERI_COULEES','PERI_AVALANCH', 'PERI_CHUTES', 'ETAGE', 'ENDOMMAGEMENT','NB_DEGAT_ARBRE']
# + données extrapolées grâce à des technologies prometteuses (satellites ?)....

add_meteo_known = ['PRCP', 'TAVE_AVG',	'TAVE', 'TAVE_GROWTH', 'PRCP_S_S',	'PRCP_G_S']

add_SAT_known = ['NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI']

In [60]:
features_past = numerics + cat_strict + cat_ord_miss
features_future = add_cat_IFN_stable + add_cat_ord_IFN_stable + add_IFN_numerics_stable + add_meteo_known + add_SAT_known

In [61]:
data_LFI1 = data_base.loc[data_base['LFI']=='LFI1',:].sort_values('PARCELLE')
data_LFI2 = data_base.loc[data_base['LFI']=='LFI2',:].sort_values('PARCELLE')
data_LFI3 = data_base.loc[data_base['LFI']=='LFI3',:].sort_values('PARCELLE')
data_LFI4 = data_base.loc[data_base['LFI']=='LFI4',:].sort_values('PARCELLE')

In [62]:
future_feat_names = []
add_cat_strict_feat_names = []
add_cat_ord_feat_names = []
add_numerics_feat_names = []

for cat in features_future:
    lfi2_list = data_LFI2[cat].to_list()
    lfi3_list = data_LFI3[cat].to_list()
    lfi4_list = data_LFI4[cat].to_list()
    data_LFI1[cat + "_f"] = lfi2_list
    data_LFI2[cat + "_f"] = lfi2_list
    data_LFI3[cat + "_f"] = lfi2_list
    future_feat_names.append(cat + '_f')
    if cat in add_cat_ord_IFN_stable:
        add_cat_ord_feat_names.append(cat + '_f')
    elif cat in add_cat_IFN_stable:
        add_cat_strict_feat_names.append(cat + '_f')
    else:
        add_numerics_feat_names.append(cat + '_f')


In [63]:
data_red = pd.concat([data_LFI1, data_LFI2, data_LFI3], axis=0)[features_past + future_feat_names]
Y = pd.concat([data_LFI2[TARGET], data_LFI3[TARGET], data_LFI4[TARGET]], axis = 0)

In [64]:
data_red.head()

,ALT,TIGES_VIV_H,SURF_TER_HA,FEUILL_PER,CONIF_PER,PERF_CROI,NDVI,EVI,NDMI,NDWI,...,TAVE_AVG_f,TAVE_f,TAVE_GROWTH_f,PRCP_S_S_f,PRCP_G_S_f,NDVI_f,EVI_f,NDMI_f,NDWI_f,DSWI_f
0,715.918970,590.0,27.79,NaN,NaN,4676,0.3779,0.0146,0.2264,-0.3325,...,8.8859,9.026806,13.84776,117.278617,51.835496,0.4992,0.0160,0.2705,-0.4653,0.6865
5,563.829759,400.0,53.38,NaN,NaN,3402,0.2861,0.0071,0.3293,-0.2327,...,8.9951,19.793490,13.83959,122.675652,55.827235,0.5552,0.0149,0.2017,-0.4864,0.6996
9,564.885846,320.0,26.70,NaN,NaN,5617,0.6749,0.0219,0.2021,-0.5865,...,9.0637,11.671380,13.39843,128.067334,61.309991,0.5633,0.0153,0.1727,-0.5002,0.6852
14,563.551602,0.0,0.00,NaN,NaN,4258,0.6092,0.0201,0.1112,-0.5143,...,8.8805,10.459870,13.15530,130.759873,63.219754,0.6059,0.0197,0.2269,-0.5588,0.8384
19,539.769096,150.0,23.32,NaN,NaN,4330,0.7039,0.0245,0.1949,-0.5983,...,8.8384,11.544840,13.16893,131.143585,62.702111,0.6012,0.0184,0.2029,-0.5486,0.7902


In [65]:
Y.head()

,TAUX_COUV_RAJ
1,2
4,6
8,6
13,6
16,5


Traitement des données catégorielles ordonnées en numériques (gestion des "-1" éventuels) :

In [66]:
feats_cat_ord = cat_ord_miss + add_cat_ord_feat_names
for cat in feats_cat_ord:
  data_red[cat] = data_red[cat].apply(lambda v : v if v!=-1 else np.nan)

LISTE DES FEATURES NUMERIQUES ET CATEGORIELLES EN VUE DU PREPROCESSING DE MODELE :

In [67]:
numerics_features = numerics + feats_cat_ord + add_numerics_feat_names
feats_cat_strict = cat_strict + add_cat_strict_feat_names

In [68]:
numerics_features

['ALT',
 'TIGES_VIV_H',
 'SURF_TER_HA',
 'FEUILL_PER',
 'CONIF_PER',
 'PERF_CROI',
 'NDVI',
 'EVI',
 'NDMI',
 'NDWI',
 'DSWI',
 'TAILLE_PPL',
 'MELANGE',
 'HT_VEG',
 'QUAL_STATION',
 'HT_VEG_f',
 'QUAL_STATION_f',
 'ALT_f',
 'PRCP_f',
 'TAVE_AVG_f',
 'TAVE_f',
 'TAVE_GROWTH_f',
 'PRCP_S_S_f',
 'PRCP_G_S_f',
 'NDVI_f',
 'EVI_f',
 'NDMI_f',
 'NDWI_f',
 'DSWI_f']